In [ ]:
import json
from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer

In [ ]:
client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [ ]:
model_name = 'all-mpnet-base-v2'

In [ ]:
history_storage = 'data/query_history.jsonl'

In [ ]:
model = SentenceTransformer(
    model_name, 
    trust_remote_code=True,
    cache_folder="./models"   # explicitly setting cache location
)
emb_dimensions = model.get_sentence_embedding_dimension()

In [ ]:
collection_name = "bfp-a3447q_context"

if client.collection_exists(collection_name):
    client.get_collection(collection_name)
    print(f"Collection {collection_name} restored")
else:
    print(f"Collection {collection_name} not found.\nCreate collection first")
    
    

In [ ]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=model.encode(query).tolist(),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [ ]:
def search_with_history(query, limit=1,history_storage = history_storage):

    results = client.query_points(
        collection_name=collection_name,
        query=model.encode(query).tolist(),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )
    record = {}
    record['query']= query
    record['ground_truth_points']=[]
    record['limit']= limit
    record['result_points_scores']=[(point.id, point.score) for point in results.points]
    # Append without reading the whole file
    with open(history_storage, "a+") as f:
        f.write(json.dumps(record) + "\n")

    return results

In [ ]:
results = search_with_history("What is the cable length of teaching pendant?", limit = 20)

In [ ]:
for index, payload in enumerate(results.points):
    print(f"{index}) Score: {payload.score}, Point id: {payload.id}\n\tMain Chapter:\t{payload.payload['main_chapter']}\n\tChapter:\t{payload.payload['chapter']}\nContent: {payload.payload['content'][:200]}")
    print("##############################################################\n")